# Initialize notebook

In [1]:
import sys

# REPO_PATH is the local git directory
REPO_PATH='/Users/Halmagyi/Documents/Programming/Python/ML/Kaggle/Planet/planet-kaggle'
sys.path.append(REPO_PATH)

import os
import numpy as np
import pandas as pd
from importlib import reload
import utils
from keras import backend as K
from keras import metrics

from skimage import io

import matplotlib.pyplot as plt
%matplotlib inline

# Here we import the py files for current project.
import utils
import models.vgglike as vl
import models.ekami as ek
import models.aframe as aframe
import models.base as base
import helpers.planet as helpers
import helpers.dfgen as gen
# from helpers.planet import PlanetData

reload(utils)
reload(vl)
reload(ek)
reload(helpers)
reload(gen)
reload(base)
reload(aframe)

DATA_ROOT=os.environ.get('DATA')
DATA_DIR='planet'
WEIGHT_ROOT=os.environ.get('WEIGHTS')
IMG_TYPE='tif'
ROOT=f'{DATA_ROOT}/{DATA_DIR}'
WEIGHT_DIR=f'{WEIGHT_ROOT}/{DATA_DIR}' # Weights are saved and loaded here
JPG_DIR = os.path.join(ROOT, 'train-jpg')
TIF_DIR = os.path.join(ROOT, 'train-tif')

Using TensorFlow backend.


In [2]:
TAGS=[
    'primary',
    'clear',
    'agriculture',
    'road',
    'water',
    'partly_cloudy',
    'cultivation',
    'habitation',
    'haze',
    'cloudy',
    'bare_ground',
    'selective_logging',
    'artisinal_mine',
    'blooming',
    'slash_burn',
    'conventional_mine',
    'blow_down']

# Create  or import dataframe for sample

In [ ]:
pld200=helpers.PlanetData(train_size=200,create=True)

In [ ]:
pld80tag1=helpers.PlanetData(train_size=80,tags=['agriculture','primary'],create=True)

In [ ]:
pld80tag1.train_df[:10]

In [ ]:
pld80tag1.train_size,pld80tag1.valid_size

# Ekami Model: modified VGG

In [5]:
ek_bc_td1=ek.EKAMI(loss_func='binary_crossentropy',target_dim=1,metrics=['accuracy',utils.k_f2])

# Flex model: very flexible CNN

In [4]:
af_td1=aframe.Flex(loss_func='binary_crossentropy',target_dim=1,metrics=['accuracy',utils.k_f2])

# Primary Rainforest

In [5]:
pld200_pr=helpers.PlanetData(train_size=200,tags=['primary'],create=True)

In [ ]:
ek_bc_td1.fit_gen(epochs=1,pdata=pld200_pr)

In [6]:
# training sample using flex model
af_td1.fit_gen(epochs=1,pdata=pld200_pr)

Epoch 1/1
200/200 [==============================] - 1140s - loss: 0.2246 - acc: 0.9489 - k_f2: 0.9840 - val_loss: 0.3569 - val_acc: 0.9797 - val_k_f2: 0.9954


In [8]:
ek_bc_td1.save_weights(pdata=pld200_pr)

In [ ]:
ek_bc_td1.predict_image(name='train_17463',image_dir='train-tif',file_ext='tif')

In [ ]:
ek_bc_td1.predict_image(name='train_23725',image_dir='train-tif',file_ext='tif')

In [ ]:
pld200tag1.train_df['prediction']=pld200tag1.train_df.image_name.apply(lambda x: ek_bc_td1.predict_image(name=x,image_dir='train-tif',file_ext='tif'))

In [ ]:
pld200tag1.train_df[:100]

In [ ]:
ek_bc_td1._weight_path(pdata=pld200tag1)

In [ ]:
os.path.isdir(ek_bc_td1._weight_path(pdata=pld200tag1))

In [ ]:
ek_bc_td1.save_weights(pdata=pld80_pr)

# Weather label predicitions

The weather labels should be mutually exclusive.
In the training data I find 40479 images and 40478 have weather labels. So only one is missing.

In [3]:
WEATHER_LABELS=['clear','partly_cloudy','haze','cloudy'] 

In [4]:
pld200_weather=helpers.PlanetData(train_size=200,tags=WEATHER_LABELS,create=True)

In [5]:
flex_weather=aframe.Flex(loss_func='categorical_crossentropy',output_activation='softmax',target_dim=4,metrics=['accuracy'])

In [ ]:
flex_weather.fit_gen(epochs=1,pdata=pld200_weather)

Epoch 1/1
110/200 [===============>..............] - ETA: 452s - loss: 0.8934 - acc: 0.6759

In [23]:
pld200_weather.train_df

,image_name,tags,vec
19055,train_19055,clear,"[1, 0, 0, 0]"
26616,train_26616,clear,"[1, 0, 0, 0]"
699,train_699,clear,"[1, 0, 0, 0]"
34523,train_34523,clear,"[1, 0, 0, 0]"
33034,train_33034,clear,"[1, 0, 0, 0]"
10834,train_10834,haze,"[0, 0, 1, 0]"
39337,train_39337,clear,"[1, 0, 0, 0]"
28127,train_28127,clear,"[1, 0, 0, 0]"
12584,train_12584,clear,"[1, 0, 0, 0]"
1815,train_1815,clear,"[1, 0, 0, 0]"


# Rare Labels

In [ ]:
labels_df=pd.read_csv[DATA_DIR+'/train.csv']

rare_labels = [l for l in label_list if labels_df[label_list].sum()[l] < 2000]